# Tarea 1

*Entrenar un modelo Lasso con GridSearchCV usando un pipeline de polinomios de segundo grado

*Graficar coeficientes "significativos"

*Revisar cuantos parámetros son mayores a 1e-2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#se buscan los datos y se muestran
ipc_path = "../modulo3/ipc.xlsx" 
market = pd.read_excel(ipc_path, index_col=0)
market.head()

,KIMBERA MF Equity,GAPB MF Equity,CEMEXCPO MF Equity,ASURB MF Equity,GFNORTEO MF Equity,KOFUBL MF Equity,AC* MF Equity,LABB MF Equity,FEMSAUBD MF Equity,OMAB MF Equity,...,MEGACPO MF Equity,GCC* MF Equity,BIMBOA MF Equity,GMEXICOB MF Equity,GCARSOA1 MF Equity,TLEVICPO MF Equity,ALFAA MF Equity,BSMXB MF Equity,PINFRA* MF Equity,MEXBOL INDEX
DATE,,,,,,,,,,,,,,,,,,,,,
2016-01-01,40.37,152.05,8.72,243.89,94.99,123.90,104.68,13.85,161.63,83.61,...,64.19,44.81,45.95,36.79,70.98,94.34,34.10,30.25,202.73,42977.50
2016-01-04,39.16,148.57,8.43,238.29,91.79,120.29,102.09,14.41,157.62,81.77,...,64.45,43.51,45.02,35.71,69.83,92.62,33.74,29.46,202.93,42113.70
2016-01-05,38.89,147.93,8.31,233.03,91.72,118.82,101.12,13.77,157.80,81.58,...,63.16,43.51,45.77,36.45,70.70,91.89,33.99,29.72,203.56,42041.68
2016-01-06,39.13,146.99,8.29,231.97,91.63,120.32,102.50,13.65,156.37,82.03,...,63.27,43.51,45.97,35.57,70.47,90.41,33.65,29.67,199.80,41691.19
2016-01-07,39.16,141.32,7.64,230.71,89.54,120.20,101.16,13.25,155.14,80.94,...,62.78,42.76,46.18,35.01,68.15,90.23,31.46,28.50,196.75,40661.57


In [3]:
Rmkt = market.pct_change().dropna()

Xall = Rmkt.drop("MEXBOL INDEX", axis=1)
Xall = Xall.join(Rmkt["MEXBOL INDEX"].shift(1)).dropna()

Xtrain = Xall.drop("MEXBOL INDEX", axis=1)
ytrain = Xall["MEXBOL INDEX"]

In [4]:
test_period = "2019-12"
Xtest, ytest = Xtrain[test_period:], ytrain[test_period:]
Xtrain, ytrain = Xtrain[:"2019-11":], ytrain[:"2019-11"]

In [16]:
#se emplea una alpha uniforme que sirva para entrenar el modelo
from numpy.random import uniform
alpha_list = uniform(0.0001, 0.000001, 300)

In [27]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

#Se usa el modelo Lasso y el polinomio de segundo grado
pipe = Pipeline([
    ("paso1", PolynomialFeatures(degree=2)),
    ("model", Lasso())
])

params = {
    "model__normalize": [True, False],
    "model__alpha": alpha_list
}


# Inicializando el modelo (entrenarlo)
gcv = GridSearchCV(estimator=pipe, param_grid= dict(params), cv=5)
gs=gcv.fit(Xtrain, ytrain)

In [18]:
gs.best_params_

{'model__alpha': 8.793573626853857e-06, 'model__normalize': False}

In [19]:
gs.predict(Xtest)

array([-1.61158561e-03, -1.01210553e-03, -7.51981779e-04,  4.63698822e-04,
       -1.20786529e-03,  8.77949622e-04,  1.36299139e-03,  9.48151725e-04,
       -1.67071071e-04,  5.30691052e-03,  1.53651469e-03,  7.05621055e-04,
        1.89740305e-03, -1.18524120e-03, -3.20145322e-03, -1.24511296e-03,
        5.85109882e-05, -1.67071071e-04,  9.10643268e-04, -1.09405592e-03,
       -2.09324592e-03,  1.69865320e-03, -1.67071071e-04,  1.86163926e-03,
       -1.05888879e-04])

In [20]:
#print(gs.best_estimator_)
#gs.best_estimator_[1].coef_

In [21]:
#se realiza un panda series con los datos obtenidos
res = pd.Series(gs.best_estimator_[1].coef_[0:35], Xtrain.columns)
res = res.reset_index().rename({"index": "asset", 0: "value"}, axis=1)

In [22]:
import plotly.express as px

In [23]:
#se genera una grafica de barras con los coeficinetes significativos y asset
px.bar(res, x="asset", y="value")

In [24]:
#se toman solo los coeficinetes significativos diferentes de cero y se grafican
coef_sig=list(filter(lambda x: x != 0, gs.best_estimator_[1].coef_))

In [25]:
px.bar(coef_sig, x=range(len(coef_sig)), y=coef_sig)

In [26]:
#se muestran los coefientes mayores a 1e-2
coef1=list(filter(lambda x: x > 1e-2, gs.best_estimator_[1].coef_))
print("Los coeficientes significativos mayores a 1e-2 son: ", len(coef1), "\nLos cuales se muestran en la sigueintes lista: \n" ,coef1)

Los coeficientes significativos mayores a 1e-2 son:  4 
Los cuales se muestran en la sigueintes lista: 
 [0.030300206593398087, 0.037185374507940544, 0.01900642904754208, 0.03538614683051349]
